# **Tarea 3 - Word Embeddings 📚**


**Instrucciones:**
- El ejercicio consiste en:
    - Responder preguntas relativas a los contenidos vistos en los vídeos y slides de las clases.
    - Implementar el método de la Word Context Matrix. 
    - Entrenar Word2Vec y FastText sobre un pequeño corpus.
    - Evaluar los embeddings obtenidos en una tarea de clasificación.

## **Preguntas teóricas 📕 (3 puntos).** ##
Para estas preguntas no es necesario implementar código, pero pueden utilizar pseudo código.

### **Parte 1: Modelos Lineales (1.5 ptos)**

Suponga que tiene un dataset de 10.000 documentos etiquetados por 4 categorías: política, deporte, negocios y otros. 

**Pregunta 1**: Diseñe un modelo lineal capaz de clasificar un documento según estas categorías donde el output sea un vector con una distribución de probabilidad con la pertenencia a cada clase. 

Especifique: representación de los documentos de entrada, parámetros del modelo, transformaciones necesarias para obtener la probabilidad de cada etiqueta y función de pérdida escogida. **(0.75 puntos)**

>El modelo consiste en un perceptrón de 1 capa, con 4 neuronas (una para cada clase). La representación de los documentos consiste en el promedio de los embedding de cada token presente en el documento. Estos embeddings se pueden conseguir de un modelo pre entrenado, y para propósitos de este ejemplo puede ser de tamaño 300.
>
>Los parámetros presentes en el modelo son la matriz de pesos $W$ y el vector de bias $b$, de tamaño 300x4 y 4 respectivamente. Para obtener una distribución de probabilidad sobre la salida se utiliza una función de activación Softmax. Finalmente, se utiliza una función de pérdidas categorical cross entropy loss y una heurística SGD para encontrar el óptimo con los datos de entrenamiento (embeddings y etiquetas)

**Pregunta 2**: Explique cómo funciona el proceso de entrenamiento en este tipo de modelos y su evaluación. **(0.75 puntos)**

>Los modelos como el diseñado previamente, requieren de un input con una dimensionalidad definida (generalmente llamadas features) y una clase o etiqueta que indica la respuesta correcta. Al entregar el input al modelo, se obtiene una respuesta que luego se compara con la respuesta correcta, que puede ser un vector de la dimensionalidad del output o un único valor numérico. En este último caso, el output que entrega el modelo se transforma, por ejemplo, al obtener el índice del número más grande (que en este caso, indica la clase más probable).
>
>Una vez que el output y la etiqueta tengan la misma dimensionalidad, se utiliza alguna función (conoocida como _loss function_) para comparar qué tan parecidas o cercanas son las respuestas. Usualmente, se recurre a funciones como MSE o Cross Entropy, aunque existen más funciones que se pueden usar como una función de pérdidas.
>
>Teóricamente, si la función de pérdidas entrega un valor igual a 0, significa que el modelo ha entregado la respuesta correcta. Generalmente este no es el caso, por lo que se usa este valor para actualizar los parámetros del modelo con el objetivo de minimizar la función de pérdidas, de modo que en una futura utilización del modelo, esté más cerca de entregar la respuesta correcta. La actualización de los parámetros se realiza por medio de alguna variante de SGD, como ADAM o RMSProp. Estos algoritmos usan el valor obtenido por la función de pérdidas para actualizar los parámetros de la última capa del modelo, y de forma recursiva retroceden por las capas del modelo actualizando parámetros gracias a la obtención de los gradientes entregados por _back propagation_.
>
>Este procedimiento de usar el modelo, obtener una estimación, calcular la distancia entre la estimación y la respuesta, y usar la distancia para actualizar parámetros se puede realizar para cada input, o de forma simultánea para un conjunto de inputs, a los que se les denomina _batch_. En caso de utilizar batches, la actualización de parámetros no se realiza de forma independiente para cada input, sino que se usa una función de agregación (como el promedio) sobre todas las distancias del batch, y este último valor es el que se utiliza para actualizar los parámetros del modelo por SGD.
>
>En cuanto a la evaluación, se suele separar un conjunto de prueba o test del conjunto con el que ha entrenado el modelo, con el objetivo de verificar que tiene la capacidad de entregar la respuesta correcta cuando se usan datos que no han sido vistos por el modelo. Dependiendo del problema, se usan distintas métricas para determinar la calidad del modelo, como el accuracy, precision, recall, f1-score, entre otros. También es usual la utilización de otro conjunto de datos (conocido como conjunto de validación), para determinar que durante el entrenamiento el modelo está efectivamente aprendiendo, en lugar de, por ejemplo, presentar sobreajuste y no tener la capacidad de inferir una respuesta para datos no vistos.

### **Parte 2: Redes Neuronales (1.5 ptos)** 

Supongamos que tenemos la siguiente red neuronal.

![Red Neuronal](../../assets/tarea3-neural-network.png)

**Pregunta 1**: En clases les explicaron como se puede representar una red neuronal de una y dos capas de manera matemática. Dada la red neuronal anterior, defina la salida $\vec{\hat{y}}$ en función del vector $\vec{x}$, pesos $W^i$, bias $b^i$ y funciones $g,f,h$. 

Adicionalmente liste y explicite las dimensiones de cada matriz y vector involucrado en la red neuronal. **(0.75 Puntos)**

>Formula:
>$$ \vec{\hat{y}} = NN_{MLP3}(\vec{x}) = g(g(g(\vec{x}W^1 + \vec{b^1})W^2 + b^2)W^3 + b^3)W^4 + b^4 $$
>
>Dimensiones:
>$$ \vec{x} \in \mathbb{R^3} \\
    W^1 \in \mathbb{R^3} \times \mathbb{R^2} \\
    \vec{b^1} \in \mathbb{R^2} \\
    W^2 \in \mathbb{R^2} \times \mathbb{R^3} \\
    \vec{b^2} \in \mathbb{R^3} \\
    W^3 \in \mathbb{R^3} \times \mathbb{R} \\
    \vec{b^3} \in \mathbb{R} \\
    W^4 \in \mathbb{R} \times \mathbb{R^4} \\
    \vec{b^4} \in \mathbb{R^4} \\ $$

**Pregunta 2**: Explique qué es backpropagation. ¿Cuales serían los parámetros a evaluar en la red neuronal anterior durante backpropagation? **(0.25 puntos)**

>Back propagation es un algoritmo que calcula el gradiente de la función de pérdidas, con el objetivo de luego actualizar los parámetros del modelo para minimizar el valor de la función de pérdidas. Para ello, se calcula el gradiente en función de los parámetros $W$ y $b$ de la capa de salida, para luego usar la regla de la cadena y así obtener el gradiente de las capas anteriores en función de los gradientes ya calculados.

**Pregunta 3**: Explique los pasos de backpropagation. En la red neuronal anterior: Cuales son las derivadas que debemos calcular para poder obtener $\vec{\delta^l_{[j]}}$ en todas las capas? **(0.5 puntos)**

>Como el nombre lo indica, backpropagation es un algoritmo que retropropaga el cálculo del gradiente a tráves de las capas del modelo. De forma general, para una capa $l$ con matriz de pesos $W$, el valor del gradiente es:
>
>$$ \frac{\partial L}{\partial W^l_{i,j}} = \frac{\partial L}{\partial h^l_{j}} \cdot \frac{\partial h^l_{j}}{\partial W^l_{i,j}} $$
>
>En donde $h^l_{j}$ es la salida de la unidad $j$ en la capa $l$, y se calcula como
>$$ h^l_{j} = \sum_{i} W^l_{i,j} \cdot g(h^{l-1}_j) $$
>
>Cada peso $b$ puede ser añadido a la sumatoria anterior si se introduce una unidad extra con un valor fijo igual a $1$. Luego, al introducir la notación delta y obtener la segunda derivada parcial de la primera expresión:
>
>$$ \delta^l_j = \frac{\partial L}{\partial h^l_{j}} $$
>$$ \frac{\partial h^l_{j}}{\partial W^l_{i,j}} = g(h^{l-1}_j) $$
>
>De modo que el gradiente resulta en
>
>$$ \frac{\partial L}{\partial W^l_{i,j}} = \delta^l_j \cdot g(h^{l-1}_j) $$
>
>En la última capa del modelo el cálculo de $\delta$ es directo, por lo que sólo se debe derivar la función de pérdidas. Para el ejemplo, serían las 4 derivadas $\frac{\partial L}{\partial h^4_{j}}$, con $j \in \{1,2,3,4\}$. Para las capas ocultas el cálculo es
>
>$$
>\begin{align*}
>\delta^l_j & = \sum_k \frac{\partial L}{\partial h^{l+1}_{k}} \cdot \frac{\partial h^{l+1}_{k}}{\partial h^l_{j}} \\
>\delta^l_j & = \sum_k \delta^{l+1}_k \cdot \frac{\partial h^{l+1}_{k}}{\partial h^l_{j}} \\
>\delta^l_j & = \sum_k \delta^{l+1}_k \cdot W^{l+1}_{j,k} \cdot g'(h^l_j) \\
>\delta^l_j & = g'(h^l_j) \sum_k \delta^{l+1}_k \cdot W^{l+1}_{j,k}
>\end{align*}
>$$
>
>En el ejemplo, la capa 3 del modelo pueder usar los valores $\delta^4$ para calcular $\delta^3$, y así retroceder hasta tener los gradientes de la primera capa.

## **Preguntas prácticas 💻 (3 puntos).** ##

### Parte 3 A (1 Punto): Word Contex Matrix



En esta parte debe crear una matriz palabra contexto, para esto, complete el siguiente template (para esta parte puede utilizar las librerías ```numpy``` y/o ```scipy```). Hint: revise como utilizar matrices sparse de ```scipy```

In [32]:
from scipy.sparse import dok_array
from typing import Callable
import numpy as np
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize

nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\esteban\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
class WordContextMatrix:
    """Clase que construye la matriz palabra contexto
    """
    def __init__(self, vocab_size: int, window_size: int, dataset: list[str], tokenizer: Callable[[str, ...], list[str]]) -> None:
        """Constructor de la clase

        Parameters
        ----------
        vocab_size : int
            tamaño máximo del vocabulario
        window_size : int
            Tamaño de la ventana contextual
        dataset : list[str]
            Conjunto de documentos a procesar
        tokenizer : Callable[[str, ...], list[str]]
            tokenizador usado sobre el dataset
        """

        # se sugiere agregar un una estructura de datos para guardar las
        # palabras del vocab y para guardar el conteo de coocurrencia
        # si lo necesita puede agregar más parametros pero no puede cambiar el resto
        self.vocab_size = vocab_size
        self.window_size = window_size
        self.dataset = dataset
        self.tokenizer = tokenizer

        self.tokenized_dataset = [self.tokenizer(doc.lower()) for doc in self.dataset]

    def build_vocab(self) -> None:
        """Construye el vocabulario a partir del dataset

        Raises
        ------
        ValueError
            Si el vocabulario del dataset es mayor que el tamaño maximo permitido
        """

        # Le puede ser útil considerar un token unk al vocab
        # para palabras fuera del vocab
        vocab = set(token for doc in self.tokenized_dataset for token in doc)
        stoi = {token: i for i, token in enumerate(vocab)}
        if len(stoi) > self.vocab_size:
            raise ValueError(f"dataset vocabulary is larger than vocab_size, {len(vocab)} > {self.vocab_size}.")

        self.vocab = list(stoi.keys())
        self.stoi = stoi

    
    def build_matrix(self) -> dok_array:
        """Construye la matriz palabra contexto a partir del dataset

        Returns
        -------
        dok_array
            matriz dispersa con los valores de la coocurrencia
        """
        self.word_context_matrix = dok_array((self.vocab_size, self.vocab_size), dtype=np.uint16)
        for doc in self.tokenized_dataset:
            for i, token in enumerate(doc):
                token_idx = self.stoi[token]
                for j in range(i + 1, min(i + self.window_size + 1, len(doc))):
                    context_idx = self.stoi[doc[j]]
                    self.word_context_matrix[token_idx, context_idx] += 1
                    self.word_context_matrix[context_idx, token_idx] += 1

        return self.word_context_matrix

    @property
    def get_matrix(self) -> dict[str, np.ndarray]:
        """Obtiene un diccionario que representa la matriz palabra contexto

        Returns
        -------
        dict[str, np.ndarray]
            Diccionario en donde cada 'key' es un token del vocabulario y el valor es el embedding del token
        """

        # se recomienda transformar la matrix a un diccionario de embedding.
        # por ejemplo {palabra1:vec1, palabra2:vec2, ...}
        return {self.vocab[i]: self.word_context_matrix[i:i+1, :len(self.stoi)].toarray().flatten() for i in self.stoi.values()}

In [3]:
corpus = [
  "I like deep learning.",
  "I like NLP.",
  "I enjoy flying."
]

wcm = WordContextMatrix(vocab_size=50, window_size=1, dataset=corpus, tokenizer=word_tokenize)
wcm.build_vocab()
sparse = wcm.build_matrix()

df = pd.DataFrame(wcm.get_matrix, index=wcm.vocab)
new_order = ["i", "like", "enjoy", "deep", "learning", "nlp", "flying", "."]
df.reindex(index=new_order, columns=new_order)

,i,like,enjoy,deep,learning,nlp,flying,.
i,0,2,1,0,0,0,0,0
like,2,0,0,1,0,1,0,0
enjoy,1,0,0,0,0,0,1,0
deep,0,1,0,0,1,0,0,0
learning,0,0,0,1,0,0,0,1
nlp,0,1,0,0,0,0,0,1
flying,0,0,1,0,0,0,0,1
.,0,0,0,0,1,1,1,0


Puede modificar los parámetros o métodos si lo considera necesario. Para probar la matrix puede utilizar el siguiente corpus.

```python
corpus = [
  "I like deep learning.",
  "I like NLP.",
  "I enjoy flying."
]
```

Obteniendo una matriz parecia a esta:

***Resultado esperado***: 

| counts   | I  | like | enjoy | deep | learning | NLP | flying | . |   
|----------|---:|-----:|------:|-----:|---------:|----:|-------:|--:|
| I        | 0  |  2   |  1    |    0 |  0       |   0 | 0      | 0|            
| like     |  2 |    0 |  0    |    1 |  0       |   1 | 0      | 0 | 
| enjoy    |  1 |    0 |  0    |    0 |  0       |   0 | 1      | 0 |
| deep     |  0 |    1 |  0    |    0 |  1       |   0 | 0      | 0 |  
| learning |  0 |    0 |  0    |    1 |  0       |   0 | 0      | 1 |          
| NLP      |  0 |    1 |  0    |    0 |  0       |   0 | 0      | 1 |
| flying   |  0 |    0 |  1    |    0 |  0       |   0 | 0      | 1 | 
| .        |  0 |    0 |  0    |    0 |  1       |   1 | 1      | 0 | 


Verifique si su matrix es igual a esta utilizando el corpus de ejemplo. Ojo que este es sólo un ejemplo, su algoritmo debe **generalizar** a otros ejemplos.

### **Parte 3 B (1 Punto): Word Embeddings**

En la auxiliar 2 aprendieron como entrenar Word2Vec utilizando gensim. El objetivo de esta parte es comparar los embeddings obtenidos con dos modelos diferentes: Word2Vec y [FastText](https://radimrehurek.com/gensim/models/fasttext.html) (utilizen size=200 en FastText) entrenados en el mismo dataset de diálogos de los Simpson. 

In [4]:
import re
from time import time
from multiprocessing import cpu_count
from sklearn.linear_model import LogisticRegression
from collections import Counter
from sklearn.metrics import confusion_matrix, classification_report

# word2vec
from gensim.models import Word2Vec, FastText
from gensim.models.phrases import Phrases, Phraser
from gensim.interfaces import TransformedCorpus
from sklearn.model_selection import train_test_split
import logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

Utilizando el dataset adjunto con la tarea:

In [5]:
data_file = "../../data/dialogue-lines-of-the-simpsons.zip"
df = pd.read_csv(data_file)
stopwords = pd.read_csv(
    'https://raw.githubusercontent.com/Alir3z4/stop-words/master/english.txt'
).values
stopwords = Counter(stopwords.flatten().tolist())
df = df.dropna().reset_index(drop=True) # Quitar filas vacias

In [6]:
df

,raw_character_text,spoken_words
0,Miss Hoover,"No, actually, it was a little of both. Sometim..."
1,Lisa Simpson,Where's Mr. Bergstrom?
2,Miss Hoover,I don't know. Although I'd sure like to talk t...
3,Lisa Simpson,That life is worth living.
4,Edna Krabappel-Flanders,The polls will be open from now until the end ...
...,...,...
131848,Miss Hoover,I'm back.
131849,Miss Hoover,"You see, class, my Lyme disease turned out to ..."
131850,Miss Hoover,Psy-cho-so-ma-tic.
131851,Ralph Wiggum,Does that mean you were crazy?


**Pregunta 1**: Ayudándose de los pasos vistos en la auxiliar, entrene los modelos Word2Vec y FastText sobre el dataset anterior. **(1 punto)** (Hint, le puede servir explorar un poco los datos)

**Respuesta**:

Uno de los pasos en el preprocesamiento del texto consiste en juntar tokens que por sí solos no representan alguna entidad, y convertirlos en bigramas. Por ejemplo "Homer Simpson" en lugar de los tokens "Homer" y "SImpson" por separado. Para estimar el umbral en que estos tokens deben aparecer para juntarlos, se realiza un conteo para saber cuántas veces aparecen en el corpus

In [7]:
word_counts = Counter(' '.join(df.raw_character_text).lower().split())
word_counts

Counter({'simpson': 66733,
         'homer': 28232,
         'marge': 13349,
         'bart': 13257,
         'lisa': 10903,
         'burns': 3157,
         'c.': 3085,
         'montgomery': 3077,
         'moe': 2877,
         'szyslak': 2810,
         'skinner': 2710,
         'flanders': 2522,
         'seymour': 2387,
         'wiggum': 2345,
         'van': 2097,
         'houten': 2089,
         'ned': 2065,
         'the': 2024,
         'grampa': 1899,
         'milhouse': 1843,
         'chief': 1828,
         'krusty': 1777,
         'clown': 1715,
         'lenny': 1188,
         'nelson': 1186,
         'muntz': 1183,
         'dr.': 1176,
         'leonard': 1151,
         'nahasapeemapetilon': 1133,
         'bouvier': 1081,
         'apu': 1003,
         'smithers': 969,
         'waylon': 965,
         'brockman': 904,
         'carl': 885,
         'kent': 885,
         'sideshow': 885,
         'man': 880,
         'announcer': 875,
         'carlson': 852,
        

In [8]:
def tokenize(text: str, stopwords: list[str]|None = None) -> list[str]:
    """Tokeniza un texto

    Parameters
    ----------
    text : str
        Texto a ser tokenizado
    stopwords : list[str] | None, opcional
        Lista de stopwords, por defecto None

    Returns
    -------
    list[str]
        Lista con los tokens del texto
    """
    # Reemplaza las apóstrofes por guiones bajos. Así, textos como "I'm" o "it's" se consideran como un token
    text_without_apos = text.replace("'", "_")
    pattern = r"\s+|[^\w\s+]" #Separa en espacios y en caracteres no palabras
    tokens = list(filter(None, re.split(pattern, text_without_apos.lower()))) # Filtra todos los 'None'

    if stopwords:
        return [token for token in tokens if token not in stopwords]
    return tokens


def phrase_unifier(tokenized_sentences: pd.Series, min_count: int = 50) -> TransformedCorpus:
    """Convierte en bigramas tokens que suelen aparecer juntos

    Parameters
    ----------
    tokenized_sentences : pd.Series
        Texto ya tokenizado
    min_count : int, opcional
        Cantidad mínima de apariciones para contar como bigrama, por defecto 50

    Returns
    -------
    TransformedCorpus
        Objeto con los bigramas encontrados en el corpus
    """
    phrases = Phrases(tokenized_sentences, min_count=min_count, progress_per=5000)
    bigram = Phraser(phrases)
    sentences = bigram[tokenized_sentences]
    return sentences

In [9]:
tokenized_sentences = df.spoken_words.apply(tokenize)
sentences = phrase_unifier(tokenized_sentences)

2024-03-05 21:32:29,793 : INFO : collecting all words and their counts
2024-03-05 21:32:29,794 : INFO : PROGRESS: at sentence #0, processed 0 words and 0 word types
2024-03-05 21:32:29,830 : INFO : PROGRESS: at sentence #5000, processed 49622 words and 33155 word types
2024-03-05 21:32:29,861 : INFO : PROGRESS: at sentence #10000, processed 97620 words and 56822 word types
2024-03-05 21:32:29,891 : INFO : PROGRESS: at sentence #15000, processed 143188 words and 76754 word types
2024-03-05 21:32:29,934 : INFO : PROGRESS: at sentence #20000, processed 197460 words and 98732 word types
2024-03-05 21:32:29,974 : INFO : PROGRESS: at sentence #25000, processed 250684 words and 119821 word types
2024-03-05 21:32:30,013 : INFO : PROGRESS: at sentence #30000, processed 307225 words and 141052 word types
2024-03-05 21:32:30,052 : INFO : PROGRESS: at sentence #35000, processed 360500 words and 159166 word types
2024-03-05 21:32:30,088 : INFO : PROGRESS: at sentence #40000, processed 408306 words 

Entrenamiento de Word2Vec

In [10]:
simpsons_w2v = Word2Vec(
    min_count=10,
    window=3,
    vector_size=200,
    sample=6e-5,
    alpha=0.03,
    min_alpha=0.0007,
    negative=20,
    workers=cpu_count()
)

2024-03-05 21:32:31,352 : INFO : Word2Vec lifecycle event {'params': 'Word2Vec<vocab=0, vector_size=200, alpha=0.03>', 'datetime': '2024-03-05T21:32:31.352491', 'gensim': '4.3.2', 'python': '3.11.7 (main, Jan  8 2024, 06:21:35) [MSC v.1929 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22621-SP0', 'event': 'created'}


In [11]:
simpsons_w2v.build_vocab(sentences, progress_per=10000)

2024-03-05 21:32:31,364 : INFO : collecting all words and their counts
2024-03-05 21:32:31,365 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2024-03-05 21:32:31,452 : INFO : PROGRESS: at sentence #10000, processed 94804 words, keeping 9927 word types
2024-03-05 21:32:31,532 : INFO : PROGRESS: at sentence #20000, processed 191865 words, keeping 14963 word types
2024-03-05 21:32:31,599 : INFO : PROGRESS: at sentence #30000, processed 298592 words, keeping 19424 word types
2024-03-05 21:32:31,668 : INFO : PROGRESS: at sentence #40000, processed 396670 words, keeping 22383 word types
2024-03-05 21:32:31,725 : INFO : PROGRESS: at sentence #50000, processed 487128 words, keeping 25140 word types
2024-03-05 21:32:31,778 : INFO : PROGRESS: at sentence #60000, processed 571614 words, keeping 27426 word types
2024-03-05 21:32:31,842 : INFO : PROGRESS: at sentence #70000, processed 666360 words, keeping 29763 word types
2024-03-05 21:32:31,908 : INFO : PROGRESS: at se

In [12]:
t = time()
simpsons_w2v.train(sentences, total_examples=simpsons_w2v.corpus_count, epochs=15, report_delay=10)
print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))

2024-03-05 21:32:32,385 : INFO : Word2Vec lifecycle event {'msg': 'training model with 16 workers on 7338 vocabulary and 200 features, using sg=0 hs=0 sample=6e-05 negative=20 window=3 shrink_windows=True', 'datetime': '2024-03-05T21:32:32.385828', 'gensim': '4.3.2', 'python': '3.11.7 (main, Jan  8 2024, 06:21:35) [MSC v.1929 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22621-SP0', 'event': 'train'}
2024-03-05 21:32:33,430 : INFO : EPOCH 0 - PROGRESS: at 50.01% examples, 232301 words/s, in_qsize 31, out_qsize 5
2024-03-05 21:32:33,668 : INFO : EPOCH 0: training on 1279712 raw words (476523 effective words) took 1.3s, 380848 effective words/s
2024-03-05 21:32:34,711 : INFO : EPOCH 1 - PROGRESS: at 46.22% examples, 210286 words/s, in_qsize 29, out_qsize 15
2024-03-05 21:32:34,922 : INFO : EPOCH 1: training on 1279712 raw words (475923 effective words) took 1.2s, 384580 effective words/s
2024-03-05 21:32:36,029 : INFO : EPOCH 2 - PROGRESS: at 50.32% examples, 219882 words/s, in_qsize 31

Time to train the model: 0.32 mins


Entrenamiento de FastText

In [13]:
simpsons_ft = FastText(
    min_count=1,
    window=3,
    vector_size=200,
    sample=6e-5,
    alpha=0.03,
    min_alpha=0.0007,
    negative=20,
    workers=cpu_count()
)

2024-03-05 21:32:51,537 : INFO : FastText lifecycle event {'params': 'FastText<vocab=0, vector_size=200, alpha=0.03>', 'datetime': '2024-03-05T21:32:51.537598', 'gensim': '4.3.2', 'python': '3.11.7 (main, Jan  8 2024, 06:21:35) [MSC v.1929 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22621-SP0', 'event': 'created'}


In [14]:
simpsons_ft.build_vocab(sentences, progress_per=10000)

2024-03-05 21:32:51,547 : INFO : collecting all words and their counts
2024-03-05 21:32:51,548 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2024-03-05 21:32:51,631 : INFO : PROGRESS: at sentence #10000, processed 94804 words, keeping 9927 word types
2024-03-05 21:32:51,749 : INFO : PROGRESS: at sentence #20000, processed 191865 words, keeping 14963 word types
2024-03-05 21:32:51,814 : INFO : PROGRESS: at sentence #30000, processed 298592 words, keeping 19424 word types
2024-03-05 21:32:51,907 : INFO : PROGRESS: at sentence #40000, processed 396670 words, keeping 22383 word types
2024-03-05 21:32:51,964 : INFO : PROGRESS: at sentence #50000, processed 487128 words, keeping 25140 word types
2024-03-05 21:32:52,024 : INFO : PROGRESS: at sentence #60000, processed 571614 words, keeping 27426 word types
2024-03-05 21:32:52,090 : INFO : PROGRESS: at sentence #70000, processed 666360 words, keeping 29763 word types
2024-03-05 21:32:52,155 : INFO : PROGRESS: at se

In [15]:
t = time()
simpsons_ft.train(sentences, total_examples=simpsons_ft.corpus_count, epochs=15, report_delay=10)
print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))

2024-03-05 21:32:55,939 : INFO : FastText lifecycle event {'msg': 'training model with 16 workers on 41472 vocabulary and 200 features, using sg=0 hs=0 sample=6e-05 negative=20 window=3 shrink_windows=True', 'datetime': '2024-03-05T21:32:55.939674', 'gensim': '4.3.2', 'python': '3.11.7 (main, Jan  8 2024, 06:21:35) [MSC v.1929 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22621-SP0', 'event': 'train'}
2024-03-05 21:32:56,965 : INFO : EPOCH 0 - PROGRESS: at 29.78% examples, 169711 words/s, in_qsize 23, out_qsize 8
2024-03-05 21:32:57,748 : INFO : EPOCH 0: training on 1279712 raw words (564226 effective words) took 1.8s, 315294 effective words/s
2024-03-05 21:32:58,943 : INFO : EPOCH 1 - PROGRESS: at 47.01% examples, 249348 words/s, in_qsize 24, out_qsize 1
2024-03-05 21:32:59,767 : INFO : EPOCH 1: training on 1279712 raw words (564565 effective words) took 1.9s, 301767 effective words/s
2024-03-05 21:33:00,924 : INFO : EPOCH 2 - PROGRESS: at 32.29% examples, 162297 words/s, in_qsize 30

Time to train the model: 0.54 mins


**Pregunta 2**: Encuentre las palabras mas similares a las siguientes: Lisa, Bart, Homer, Marge. Cúal es la diferencia entre ambos resultados? Por qué ocurre esto? Intente comparar ahora Liisa en ambos modelos (doble i). Cuando escogería uno vs el otro? **(0.5 puntos)**

**Respuesta**:

In [16]:
def most_similar(word: str, w2v: Word2Vec, ft: FastText) -> pd.DataFrame:
    """Compara las 10 palabras más similares a 'word' entre Word2Vec y FastText

    Parameters
    ----------
    word : str
        Palabra a la que se le desea encontrar las palabras más similares
    w2v : Word2Vec
        Word2Vec entrenado
    ft : FastText
        FastText entrenado

    Returns
    -------
    pd.DataFrame
        DataFrame con los resultados más similares
    """
    columns = [("Word2Vec", "Word"), ("Word2Vec", "Value"), ("FastText", "Word"), ("FastText", "Value")]
    w2v_list = w2v.wv.most_similar(positive=[word])
    ft_list = ft.wv.most_similar(positive=[word])
    w2v_words, w2v_values = zip(*w2v_list)
    ft_words, ft_values = zip(*ft_list)
    comp_df = pd.DataFrame(zip(w2v_words, w2v_values, ft_words, ft_values), columns=columns, index=range(1,11))
    comp_df.columns = pd.MultiIndex.from_tuples(comp_df.columns)
    return comp_df

In [17]:
most_similar("lisa", simpsons_w2v, simpsons_ft)

Word2Vec               FastText          
           Word     Value         Word     Value
1          bart  0.827519          lis  0.881775
2        maggie  0.802026        lisui  0.872436
3       sweetie  0.759638  little_girl  0.792820
4           mom  0.732529          mom  0.761338
5           ned  0.731611        womie  0.731402
6   your_father  0.715023        honey  0.731129
7         marge  0.713937          lie  0.731030
8      milhouse  0.711964          apu  0.726771
9         honey  0.710189          dad  0.723257
10         alex  0.706531    milhousey  0.716130

In [18]:
most_similar("bart", simpsons_w2v, simpsons_ft)

Word2Vec              FastText          
           Word     Value        Word     Value
1          lisa  0.827519      bart_d  0.861538
2           mom  0.804480    bartdude  0.851226
3        maggie  0.770294      baaart  0.837848
4           dad  0.766192       barto  0.834909
5      milhouse  0.731611        dart  0.833009
6   your_father  0.730824       kvart  0.830519
7         homie  0.721960  beelzebart  0.823468
8         youse  0.703928        kart  0.812005
9         homer  0.700669      mozart  0.805260
10        bongo  0.693523     bartolo  0.797270

In [19]:
most_similar("homer", simpsons_w2v, simpsons_ft)

Word2Vec               FastText          
          Word     Value         Word     Value
1        marge  0.823189     homelier  0.926906
2          ned  0.750561    homemaker  0.862293
3          dad  0.745015  homewrecker  0.851214
4          abe  0.739446      hooomer  0.817440
5        homie  0.735338        gomer  0.800946
6   sweetheart  0.730065     misnomer  0.795923
7      ralphie  0.711727     customer  0.794764
8        bongo  0.707165     awesomer  0.794013
9       joking  0.706133        homeo  0.778768
10        bart  0.700669      groomer  0.776997

In [20]:
most_similar("marge", simpsons_w2v, simpsons_ft)

Word2Vec            FastText          
          Word     Value      Word     Value
1        homer  0.823189   marge_d  0.929213
2        homie  0.797105    _marge  0.886489
3          abe  0.747796    maarge  0.872063
4        bongo  0.747299  margaret  0.829152
5   sweetheart  0.741278  marriott  0.824199
6        becky  0.736347   marriot  0.823181
7          dad  0.733117    margie  0.819057
8          ned  0.731311   maaarge  0.813767
9         lisa  0.713937    margin  0.810192
10       greta  0.713150       ma_  0.797185

In [21]:
most_similar("liisa", simpsons_w2v, simpsons_ft)

KeyError: "Key 'liisa' not present in vocabulary"

In [22]:
simpsons_ft.wv.most_similar(positive=["liisa"])

[('lisui', 0.9513565301895142),
 ('lieu', 0.9166699647903442),
 ('liyã', 0.9128382802009583),
 ('liugi', 0.9084550142288208),
 ('liz', 0.9067040085792542),
 ('liii', 0.8914716839790344),
 ('liza', 0.8842250108718872),
 ('lisa', 0.8714575171470642),
 ('liiiiisa', 0.8702735900878906),
 ('li', 0.8644471764564514)]

FastText suele tener mejor rendimiento en tareas sintácticas, mientras que Word2Vec puede entregar mejor performance en tareas semánticas. Además, FastText tiene una ligera ventaja cuando el conjunto de entrenamiento es pequeño, aunque estas diferencias se acortan a medida que se tienen más datos de entrenamiento. Fuente: [Word2Vec_FastText_Comparison.ipynb](https://github.com/piskvorky/gensim/blob/develop/docs/notebooks/Word2Vec_FastText_Comparison.ipynb)

Además de lo anterior, FastText tiene la ventaja de ser capaz de obtener embedding de palabras fuera del vocabulario, de modo que no ocurre lo mismo que en el ejemplo de "liisa". Si existe alguna tarea en la que obtener un vocabulario extenso durante entrenamiento es un problema, se puede usar FastText para evitar tener embedding vacios o poco representativos de cada documento en inferencia.

También se debe tomar en cuenta que FastText busca subwords, y como se ha apreciado en los resultados anteriores las palabras más cercanas no son en realidad palabras, sino subpalabras.

### **Parte 4 (1 Punto): Aplicar embeddings para clasificar**

Ahora utilizaremos los embeddings que acabamos de calcular para clasificar palabras basadas en su polaridad (positivas o negativas). 

Para esto ocuparemos el lexicón AFINN incluido en la tarea, que incluye una lista de palabras y un 1 si su connotación es positiva y un -1 si es negativa.

In [23]:
AFINN = '../../data/AFINN_full.csv'
df_afinn = pd.read_csv(AFINN, sep='\t', header=None)

Hint: Para w2v son esperables KeyErrors debido a que no todas las palabras del corpus de los simpsons tendrán una representación en AFINN. Pueden utilizar esta función auxiliar para filtrar las filas en el dataframe que no tienen embeddings (como w2v no tiene token UNK se deben ignorar).

In [24]:
def try_apply(model, word):
    try:
        embedding = model.wv[word]
        return embedding
    except KeyError:
        #logger.error('Word {} not in dictionary'.format(word))
        return np.random.random(200)

**Pregunta 1**: Transforme las palabras del corpus de AFINN a la representación en embedding que acabamos de calcular (con ambos modelos). 

Su dataframe final debe ser del estilo [embedding, sentimiento], donde los embeddings corresponden a $X$ y el sentimiento asociado con el embedding a $y$ (positivo/negativo, 1/-1). 

Para ambos modelos, separar train y test de acuerdo a la siguiente función. **(0.75 puntos)**

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.1, stratify=y)

**Respuesta**:

In [25]:
get_embds = lambda word: try_apply(simpsons_w2v, word)  # noqa: E731

X_w2v = np.array(df_afinn[0].apply(func=get_embds).to_list())
y_w2v = df_afinn[1]
X_ft = np.array(df_afinn[0].apply(lambda word: simpsons_ft.wv[word]).to_list())
y_ft = df_afinn[1]

In [26]:
X_w2v_train, X_w2v_test, y_w2v_train, y_w2v_test = train_test_split(X_w2v, y_w2v, random_state=42, test_size=0.1, stratify=y_w2v)
X_ft_train, X_ft_test, y_ft_train, y_ft_test = train_test_split(X_ft, y_ft, random_state=42, test_size=0.1, stratify=y_ft)

**Pregunta 2**: Entrenar una regresión logística (vista en auxiliar) y reportar accuracy, precision, recall, f1 y confusion_matrix para ambos modelos. Por qué se obtienen estos resultados? Cómo los mejorarías? **(0.75 puntos)**

**Respuesta**:

In [27]:
clf_w2v = LogisticRegression(max_iter=1000000)
clf_w2v.fit(X_w2v_train, y_w2v_train)


clf_ft = LogisticRegression(max_iter=1000000)
clf_ft.fit(X_ft_train, y_ft_train)

LogisticRegression(max_iter=1000000)

In [28]:
y_w2v_pred = clf_w2v.predict(X_w2v_test)
conf_matrix_w2v = confusion_matrix(y_w2v_test, y_w2v_pred)
print(conf_matrix_w2v)

[[193  28]
 [ 88  30]]


In [29]:
y_ft_pred = clf_ft.predict(X_ft_test)
conf_matrix_ft = confusion_matrix(y_ft_test, y_ft_pred)
print(conf_matrix_ft)

[[192  29]
 [ 72  46]]


In [30]:
print(classification_report(y_w2v_test, y_w2v_pred))

              precision    recall  f1-score   support

          -1       0.69      0.87      0.77       221
           1       0.52      0.25      0.34       118

    accuracy                           0.66       339
   macro avg       0.60      0.56      0.55       339
weighted avg       0.63      0.66      0.62       339



In [31]:
print(classification_report(y_ft_test, y_ft_pred))

              precision    recall  f1-score   support

          -1       0.73      0.87      0.79       221
           1       0.61      0.39      0.48       118

    accuracy                           0.70       339
   macro avg       0.67      0.63      0.63       339
weighted avg       0.69      0.70      0.68       339



Las métricas globales favorecen al modelo de FastText, aunque la diferencia entre ambos no resulta considerable. Es difícil dilucidar sobre los motivos que lleva a un model oser mejor sobre otro, cuando el conjunto de testeo posee menos de 500 elementos y podría no ser representativo, se ha probado solamente un clasificador y sin variar los hiperparámetros. Como se ha mencionado anteriormente, FastText debiese rendir mejor ante datasets con pocos datos, aunque en una tarea semántica Word2Vec debiese entregar mejores resultados. Es posible que las palabras que no están presentes en el corpus y a las que se ha asignado un embedding aleatorio jueguen un rol en las métricas, desfavoreciendo a Word2Vec.

# Bonus: +0.25 puntos en cualquier pregunta

**Pregunta 1**: Replicar la parte anterior utilizando embeddings pre-entrenados en un dataset más grande y obtener mejores resultados. Les puede servir [ésta](https://radimrehurek.com/gensim/downloader.html#module-gensim.downloader) documentacion de gensim **(0.25 puntos)**.

**Respuesta**: